### Este projeto é da autoria de [José Gama](https://github.com/mixmaxze).
 O objetivo é limpar e fazer uma análise exploratória dos dados referentes aos preços dos medicamentos que circulam pelo Brasil.<br>
 Esses dados estão disponíveis no site do governo. Você os encontra [aqui](https://dados.gov.br/dataset/ef3d5f8b-1661-4381-a369-4aed487bf313/resource/84531c98-b162-4c00-ae21-002192a51051).
 

In [1]:
# Importando bibliotecas

import pandas as pd
from unidecode import unidecode

In [2]:
# Carregando o DataFrame

df = pd.read_csv('dados/TA_PRECO_MEDICAMENTO.csv', sep=';', low_memory=False)

In [3]:
# Todas os atributos disponíveis no dataset

df.columns

Index(['SUBSTÂNCIA', 'CNPJ', 'LABORATÓRIO', 'CÓDIGO GGREM', 'REGISTRO',
       'EAN 1', 'EAN 2', 'EAN 3', 'PRODUTO', 'APRESENTAÇÃO',
       'CLASSE TERAPÊUTICA', 'TIPO DE PRODUTO (STATUS DO PRODUTO)',
       'REGIME DE PREÇO', 'PF Sem Impostos', 'PF 0%', 'PF 12%', 'PF 17%',
       'PF 17% ALC', 'PF 17,5%', 'PF 17,5% ALC', 'PF 18%', 'PF 18% ALC',
       'PF 20%', 'PMC 0%', 'PMC 12%', 'PMC 17%', 'PMC 17% ALC', 'PMC 17,5%',
       'PMC 17,5% ALC', 'PMC 18%', 'PMC 18% ALC', 'PMC 20%',
       'RESTRIÇÃO HOSPITALAR', 'CAP', 'CONFAZ 87', 'ICMS 0%',
       'ANÁLISE RECURSAL',
       'LISTA DE CONCESSÃO DE CRÉDITO TRIBUTÁRIO (PIS/COFINS)',
       'COMERCIALIZAÇÃO 2019', 'TARJA'],
      dtype='object')

### Inserir link para pdf com descrição dos atributos AQUI!

In [23]:
data.head

<bound method NDFrame.head of                   SUBSTÂNCIA                CNPJ  ... COMERCIALIZAÇÃO 2019                            TARJA
0      ACETATO DE LANREOTIDA  07.718.721/0001-80  ...                  Não                       Tarja -(*)
1                  ACICLOVIR  45.992.062/0001-65  ...                  Não                Tarja Vermelha(*)
2                  ACICLOVIR  45.992.062/0001-65  ...                  Não                Tarja Vermelha(*)
3                  ACICLOVIR  45.992.062/0001-65  ...                  Não                Tarja Vermelha(*)
4         ACTAEA RACEMOSA L.  78.950.011/0001-20  ...                  Não                       Tarja -(*)
...                      ...                 ...  ...                  ...                              ...
25986              ZOPICLONA  02.685.377/0001-57  ...                  Sim  Tarja Venda Livre/Sem Tarja (*)
25987          ZUCLOPENTIXOL  04.522.600/0001-70  ...                  Sim                  Tarja  Vermelh

## Limpeza dos dados
1. Renomeando as colunas: <br>
Para tornar mais simples o acesso às colunas, irei renomeá-las com letras minúsculas e sem acentos. <br>Depois, irei abreviar algumas colunas com termos menores e remover as que não serão usadas.

In [4]:
df.columns = [str.lower(x) for x in df.columns] # deixando as letras minúsculas
df.columns = [str.replace(x, ' ','_') for x in df. columns] # substituindo espaços por underline
df.columns = [unidecode(x) for x in df.columns] # retirando os acentos
df.columns

Index(['substancia', 'cnpj', 'laboratorio', 'codigo_ggrem', 'registro',
       'ean_1', 'ean_2', 'ean_3', 'produto', 'apresentacao',
       'classe_terapeutica', 'tipo_de_produto_(status_do_produto)',
       'regime_de_preco', 'pf_sem_impostos', 'pf_0%', 'pf_12%', 'pf_17%',
       'pf_17%_alc', 'pf_17,5%', 'pf_17,5%_alc', 'pf_18%', 'pf_18%_alc',
       'pf_20%', 'pmc_0%', 'pmc_12%', 'pmc_17%', 'pmc_17%_alc', 'pmc_17,5%',
       'pmc_17,5%_alc', 'pmc_18%', 'pmc_18%_alc', 'pmc_20%',
       'restricao_hospitalar', 'cap', 'confaz_87', 'icms_0%',
       'analise_recursal',
       'lista_de_concessao_de_credito_tributario_(pis/cofins)',
       'comercializacao_2019', 'tarja'],
      dtype='object')

In [11]:
df.rename(columns={'laboratorio': 'lab', 'tipo_de_produto_(status_do_produto)': 'status_produto',
                  'regime_de_preco': 'regime_preco', 'lista_de_concessao_de_credito_tributario_(pis/cofins)': 'pis/cofins'},
                 inplace = True)
df.columns = [str.replace(x, '%', '') for x in df.columns] # removendo os símbolos de porcentagem
df.columns = [str.replace(x, ',', '.') for x in df.columns] # trocando vírgulas por pontos
df.columns

Index(['substancia', 'cnpj', 'lab', 'codigo_ggrem', 'registro', 'ean_1',
       'ean_2', 'ean_3', 'produto', 'apresentacao', 'classe_terapeutica',
       'status_produto', 'regime_preco', 'pf_sem_impostos', 'pf_0', 'pf_12',
       'pf_17', 'pf_17_alc', 'pf_17.5', 'pf_17.5_alc', 'pf_18', 'pf_18_alc',
       'pf_20', 'pmc_0', 'pmc_12', 'pmc_17', 'pmc_17_alc', 'pmc_17.5',
       'pmc_17.5_alc', 'pmc_18', 'pmc_18_alc', 'pmc_20',
       'restricao_hospitalar', 'cap', 'confaz_87', 'icms_0',
       'analise_recursal', 'pis/cofins', 'comercializacao_2019', 'tarja'],
      dtype='object')

In [12]:
df.drop(columns=['analise_recursal', 'cnpj', 'comercializacao_2019', 'codigo_ggrem', 'registro', 'confaz_87'], inplace=True)
df.columns

Index(['substancia', 'lab', 'ean_1', 'ean_2', 'ean_3', 'produto',
       'apresentacao', 'classe_terapeutica', 'status_produto', 'regime_preco',
       'pf_sem_impostos', 'pf_0', 'pf_12', 'pf_17', 'pf_17_alc', 'pf_17.5',
       'pf_17.5_alc', 'pf_18', 'pf_18_alc', 'pf_20', 'pmc_0', 'pmc_12',
       'pmc_17', 'pmc_17_alc', 'pmc_17.5', 'pmc_17.5_alc', 'pmc_18',
       'pmc_18_alc', 'pmc_20', 'restricao_hospitalar', 'cap', 'icms_0',
       'pis/cofins', 'tarja'],
      dtype='object')